In [23]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
# Set working directory equal to location of current notebook file
os.getcwd()
working_directory = os.path.dirname(os.path.abspath(__vsc_ipynb_file__))
os.chdir(working_directory)
os.getcwd()

'/home/rory/Dev/AIML/practice/deep-learning-with-pytorch'

In [25]:
# Convert mnist image files into tensor of 4 dimensions (# of images, height, width, and color channel)
transform = transforms.ToTensor()

In [26]:
# Get train data
train_data = datasets.MNIST(root=rf'./input/cnn_data', train=True, download=True, transform=transform)

In [27]:
# Get test data
test_data = datasets.MNIST(root=rf'./input/cnn_data', train=False, download=True, transform=transform)

In [28]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./input/cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [29]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./input/cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [30]:
# Create a small batch size for images ... let's say 10
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(train_data, batch_size=10, shuffle=False)

In [31]:
# Define our CNN Model
# Describe convolutional layer and what it's doing (2 conv layers)
# This is just an example
conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1)

In [32]:
# Grab 1 MNIST record/image
for i, (X_train, y_train) in enumerate(train_data):
    break

In [33]:
X_train.shape

torch.Size([1, 28, 28])

In [34]:
x = X_train.view(1,1,28,28)

In [35]:
# Perform our first convolution
x = F.relu(conv1(x)) # Rectified linear unit for our activation function

In [36]:
x

tensor([[[[0.0197, 0.0197, 0.0197,  ..., 0.0197, 0.0197, 0.0197],
          [0.0197, 0.0197, 0.0197,  ..., 0.0197, 0.0197, 0.0197],
          [0.0197, 0.0197, 0.0197,  ..., 0.0197, 0.0197, 0.0197],
          ...,
          [0.0197, 0.0197, 0.0000,  ..., 0.0197, 0.0197, 0.0197],
          [0.0197, 0.0197, 0.0146,  ..., 0.0197, 0.0197, 0.0197],
          [0.0197, 0.0197, 0.0197,  ..., 0.0197, 0.0197, 0.0197]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [37]:
# 1 single image, 6 is the filters we asked for, 26x26 is now the image because padding wasn't used and 2 were dropped around the edges
x.shape

torch.Size([1, 6, 26, 26])

In [38]:
# Pass through the pooling layer
x = F.max_pool2d(x,2,2) # Kernel of 2 and stride of 2

In [39]:
x.shape # 25 / 2 = 13

torch.Size([1, 6, 13, 13])

In [40]:
# Do our second convolutional layer
x = F.relu(conv2(x))

In [41]:
x.shape # Again, we didn't set padding and we lose 2 pixels around the outside

torch.Size([1, 16, 11, 11])

In [42]:
# Pooling layer
x = F.max_pool2d(x,2,2)

In [43]:
x.shape # 11 / 2 = 5.5 but we have to round down, because we can't invent data to round up

torch.Size([1, 16, 5, 5])

In [44]:
((28-2) / 2 - 2) / 2

5.5

In [45]:
# Model class
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=3, stride=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, stride=1)
        # Fully connected layers
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # Forward function
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2) # 2x2 kernel with stride of 2
        # Second Pass
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2) # 2x2 kernel with stride of 2

        #Re-View to flatten it out
        x = X.view(-1, 16*5*5) # Negative one so that we can vary the batch size

        # Fully connected layers
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(x))
        X = self.fc3(X)

        return F.log_softmax(X, dim=1)

In [46]:
# Create an Instance of our Model
torch.manual_seed(41)
model = ConvolutionalNetwork()
model

ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [47]:
# Loss function optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # Smaller the learning rate, the longer it will take to train... Too big can cause instability